In [ ]:
!pip install scikit_learn
!pip install transformers[torch]
!pip install accelerate>={ACCELERATE_MIN_VERSION}
!pip install pandas torch transformers scikit-learn

DEPRECATION: shuup 3.1.0 has a non-standard dependency specifier toml<1pytz>=2015.4,>=0.10.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of shuup or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: shuup 3.1.0 has a non-standard dependency specifier toml<1pytz>=2015.4,>=0.10.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of shuup or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


^C


DEPRECATION: shuup 3.1.0 has a non-standard dependency specifier toml<1pytz>=2015.4,>=0.10.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of shuup or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import Libraries


In [39]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EvalPrediction
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


Set Environment Variable

In [40]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

Define Compute Metrics Function

In [41]:
def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # Get the predicted class indices

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')  # or 'macro' based on your preference
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

Load the Dataset

In [46]:
# Load the dataset
df = pd.read_csv('reviews-kz.csv')

# Map text labels to numerical labels
label_map = {'positive': 2, 'neutral': 1, 'negative': 0}
df['label'] = df['label'].map(label_map)

# Show the first few rows
print(df.head())


   id                                               text  label
0   1                          Бул фильм оте жаксы болды      2
1   2                         Маган бул фильм унаган жок      0
2   3  Фильмнин басы кызыкты болды бирак соны тусиниксиз      1
3   4                          Мен буган оте куаныштымын      2
4   5            Бул мен куткеннен әлдекайда нашар болды      0


Load BERT Tokenizer and Model



In [ ]:
# Load the multilingual BERT tokenizer and model
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create Custom Dataset Class



In [ ]:
class KazakhSentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


Prepare the Dataset



In [47]:
# Prepare dataset
texts = df['text'].tolist()
labels = df['label'].tolist()

# Create dataset and split it into training and validation sets
dataset = KazakhSentimentDataset(texts, labels, tokenizer)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


Define Training Arguments

In [49]:
training_args = TrainingArguments(
    output_dir='C:/Users/Toktarla/OneDrive/Desktop/sentiment-analysis-kz/results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='C:/Users/Toktarla/OneDrive/Desktop/sentiment-analysis-kz/logs',
    logging_steps=10,
    eval_strategy="epoch",
)

Create Trainer and Train the Model

In [50]:
# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


  0%|          | 0/108 [00:00<?, ?it/s]

{'loss': 1.0918, 'grad_norm': 8.828658103942871, 'learning_rate': 1e-05, 'epoch': 0.28}
{'loss': 0.9232, 'grad_norm': 5.253421783447266, 'learning_rate': 2e-05, 'epoch': 0.56}
{'loss': 0.7923, 'grad_norm': 7.462895393371582, 'learning_rate': 3e-05, 'epoch': 0.83}


  0%|          | 0/9 [00:00<?, ?it/s]

c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.7051713466644287, 'eval_accuracy': 0.704225352112676, 'eval_f1': 0.6719912973777624, 'eval_precision': 0.6485715914922734, 'eval_recall': 0.704225352112676, 'eval_runtime': 19.9934, 'eval_samples_per_second': 3.551, 'eval_steps_per_second': 0.45, 'epoch': 1.0}
{'loss': 0.6787, 'grad_norm': 6.217649459838867, 'learning_rate': 4e-05, 'epoch': 1.11}
{'loss': 0.6681, 'grad_norm': 6.977437496185303, 'learning_rate': 5e-05, 'epoch': 1.39}
{'loss': 0.5157, 'grad_norm': 31.668140411376953, 'learning_rate': 4.1379310344827587e-05, 'epoch': 1.67}
{'loss': 0.356, 'grad_norm': 6.615902423858643, 'learning_rate': 3.275862068965517e-05, 'epoch': 1.94}


...

{'eval_loss': 0.6746416687965393, 'eval_accuracy': 0.8169014084507042, 'eval_f1': 0.7895947111238861, 'eval_precision': 0.7673448095983307, 'eval_recall': 0.8169014084507042, 'eval_runtime': 19.698, 'eval_samples_per_second': 3.604, 'eval_steps_per_second': 0.457, 'epoch': 2.0}
{'loss': 0.6764, 'grad_norm': 45.71118927001953, 'learning_rate': 2.413793103448276e-05, 'epoch': 2.22}
{'loss': 0.2744, 'grad_norm': 0.6593054533004761, 'learning_rate': 1.5517241379310346e-05, 'epoch': 2.5}
{'loss': 0.2342, 'grad_norm': 5.94890022277832, 'learning_rate': 6.896551724137932e-06, 'epoch': 2.78}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.6780774593353271, 'eval_accuracy': 0.8450704225352113, 'eval_f1': 0.8143980942511255, 'eval_precision': 0.7859104984517031, 'eval_recall': 0.8450704225352113, 'eval_runtime': 20.0638, 'eval_samples_per_second': 3.539, 'eval_steps_per_second': 0.449, 'epoch': 3.0}
{'train_runtime': 982.6202, 'train_samples_per_second': 0.861, 'train_steps_per_second': 0.11, 'train_loss': 0.6001380306703074, 'epoch': 3.0}


c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=108, training_loss=0.6001380306703074, metrics={'train_runtime': 982.6202, 'train_samples_per_second': 0.861, 'train_steps_per_second': 0.11, 'total_flos': 55648487849472.0, 'train_loss': 0.6001380306703074, 'epoch': 3.0})

Evaluate the Model

In [51]:
# Evaluate the model on the validation set
eval_result = trainer.evaluate()

# Access the evaluation results
accuracy = eval_result.get('eval_accuracy', eval_result.get('accuracy'))
f1 = eval_result.get('eval_f1', eval_result.get('f1'))
precision = eval_result.get('eval_precision', eval_result.get('precision'))
recall = eval_result.get('eval_recall', eval_result.get('recall'))

print(eval_result)
print(f"Validation Accuracy: {accuracy}")
print(f"Validation F1 Score: {f1}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.6780774593353271, 'eval_accuracy': 0.8450704225352113, 'eval_f1': 0.8143980942511255, 'eval_precision': 0.7859104984517031, 'eval_recall': 0.8450704225352113, 'eval_runtime': 21.2201, 'eval_samples_per_second': 3.346, 'eval_steps_per_second': 0.424, 'epoch': 3.0}
Validation Accuracy: 0.8450704225352113
Validation F1 Score: 0.8143980942511255
Validation Precision: 0.7859104984517031
Validation Recall: 0.8450704225352113


c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Define Prediction Function


In [52]:
def predict_sentiment(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Predict
    output = model(input_ids, attention_mask=attention_mask)
    logits = output.logits
    prediction = torch.argmax(logits, dim=1).item()

    label_map_reverse = {v: k for k, v in label_map.items()}
    return label_map_reverse[prediction]


Test Prediction with Sample Text


In [63]:
# Test with a new text
sample_text = "Жумыс барысында сынбайтын рух керек"
print(f"Predicted Sentiment: {predict_sentiment(sample_text)}")


Predicted Sentiment: positive
